In [1]:
!pip install -q ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 362 (delta 70), reused 70 (delta 70), pack-reused 278 (from 1)
Receiving objects: 100% (362/362), 95.27 KiB | 768.00 KiB/s, done.
Resolving deltas: 100% (210/210), done.
Filtering content: 100% (3/3), 34.77 MiB | 25.54 MiB/s, done.


In [2]:
%cd TransNetV2/inference

/content/TransNetV2/inference


In [3]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg # dùng để xử lý đa phương tiện (âm thanh, video)
import torch
import numpy as np
from tqdm import tqdm
from transnetv2 import TransNetV2

In [4]:
model = TransNetV2()

[TransNetV2] Using weights from /content/TransNetV2/inference/transnetv2-weights/.


In [5]:
save_dir = 'SceneJSON'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [6]:
!gdown --id 1crJ8Js7fzpWXQl8adZ1hPjoeJg1s9bfq

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1crJ8Js7fzpWXQl8adZ1hPjoeJg1s9bfq
From (redirected): https://drive.google.com/uc?id=1crJ8Js7fzpWXQl8adZ1hPjoeJg1s9bfq&confirm=t&uuid=ec921de0-0090-4b5f-bacd-7216392036fa
To: /content/TransNetV2/inference/L01_V001.mp4
100% 162M/162M [00:06<00:00, 25.1MB/s]


In [7]:
video_path = "L01_V001.mp4"
# Dự đoán cho từng khung hình, thể hiện xác suất của khung hình đó là một phần của cảnh chuyển tiếp hoặc không
_, single_frame_predictions, _ = model.predict_video(video_path)

# Từ xác suất bên trên biến thành những scenes và trả về một array chứa các index của frame
scenes = model.predictions_to_scenes(single_frame_predictions)
scenes

[TransNetV2] Extracting frames from L01_V001.mp4
[TransNetV2] Processing video frames 27714/27714


array([[    0,    43],
       [   44,   321],
       [  322,   376],
       [  377,   412],
       [  413,   442],
       [  443,   470],
       [  471,   503],
       [  504,   529],
       [  530,   559],
       [  560,   593],
       [  594,   627],
       [  628,   664],
       [  665,   695],
       [  696,   722],
       [  723,   752],
       [  753,   832],
       [  833,   958],
       [  959,  1727],
       [ 1728,  1783],
       [ 1784,  1844],
       [ 1845,  1893],
       [ 1894,  1941],
       [ 1942,  1986],
       [ 1987,  2044],
       [ 2045,  2108],
       [ 2109,  2172],
       [ 2173,  2234],
       [ 2235,  2277],
       [ 2278,  2327],
       [ 2328,  2378],
       [ 2379,  2449],
       [ 2450,  2504],
       [ 2505,  2558],
       [ 2559,  2600],
       [ 2601,  2655],
       [ 2656,  3223],
       [ 3224,  3300],
       [ 3301,  3330],
       [ 3331,  3352],
       [ 3353,  3508],
       [ 3509,  3649],
       [ 3650,  3778],
       [ 3779,  3920],
       [ 39

In [9]:
from get_key_frame import KeyFrames
key_frames = KeyFrames(scenes, video_path)
key_frames_idx = key_frames['middle']
key_frames_idx

[21,
 182,
 349,
 394,
 427,
 456,
 487,
 516,
 544,
 576,
 610,
 646,
 680,
 709,
 737,
 792,
 895,
 1343,
 1755,
 1814,
 1869,
 1917,
 1964,
 2015,
 2076,
 2140,
 2203,
 2256,
 2302,
 2353,
 2414,
 2477,
 2531,
 2579,
 2628,
 2939,
 3262,
 3315,
 3341,
 3430,
 3579,
 3714,
 3849,
 3974,
 4065,
 4130,
 4195,
 4260,
 4317,
 4615,
 4916,
 4982,
 5045,
 5111,
 5158,
 5199,
 5241,
 5277,
 5449,
 5613,
 5657,
 5735,
 5817,
 5887,
 5970,
 6037,
 6081,
 6121,
 6155,
 6191,
 6365,
 6555,
 6602,
 6635,
 6671,
 6712,
 6761,
 6813,
 6864,
 6925,
 7006,
 7088,
 7351,
 7743,
 7939,
 7990,
 8051,
 8132,
 8202,
 8265,
 8558,
 8844,
 8900,
 8997,
 9102,
 9168,
 9242,
 9309,
 9354,
 9469,
 9583,
 9628,
 9686,
 9748,
 9816,
 9881,
 9942,
 9983,
 10021,
 10067,
 10109,
 10290,
 10534,
 10652,
 10717,
 10785,
 10849,
 10920,
 11005,
 11070,
 11121,
 11177,
 11357,
 11547,
 11603,
 11669,
 11747,
 11794,
 11862,
 11980,
 12107,
 12186,
 12258,
 12356,
 12423,
 12467,
 12562,
 12881,
 13162,
 13248,
 13349

In [14]:
from get_key_frame_imgs import extract_frames
import os
output_folder = '/content/drive/MyDrive/Data'
os.makedirs(output_folder, exist_ok=True)


In [15]:
extract_frames(video_path, output_folder, key_frames_idx)